# Train yolov3 model on 6 classes: kick, punch, elbow, knee, clinch, stance

The following code was originally run in google colab and was adapted from [this](https://www.youtube.com/watch?v=_FNfRtXEbr4&amp%3Bab_channel=Pysource) resource.

Don't forget to check that the class labels are correct in the corresponding .txt files

In [ ]:
#check if NVIDIA GPU is enabled
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

Mon Nov 30 08:11:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#connect google colab with google drive.
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

## Clone the Darknet repo

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 14518 (delta 0), reused 3 (delta 0), pack-reused 14500
Receiving objects: 100% (14518/14518), 13.23 MiB | 3.92 MiB/s, done.
Resolving deltas: 100% (9861/9861), done.


## Compile Darknet using Nvidia GPU

In [ ]:
#change makefile to have GPU and OPENCV enabled
%cd darknet

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

## Configure Darknet network for training YOLO V3

In [ ]:
#make a copy of the confiuration file to make changes to
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

## Make the following changes:
- batch = 64

- subdivisions = 16 *(if 16 gives a CUDA out of memory error, can go up to 32)*

- max_batches = 2000*(num_classes) = 12000

- steps = (80%)\*max_batches, (90%)*max_batches = 9600, 10800

- classes = 6 *(on lines 610, 696, 783)*

- filters = (num_classes + 5)\*3 = 33 *(on lines 603, 689, 776)*



In [ ]:
#change the configuration settings
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches=500200/max_batches=12000/' cfg/yolov3_training.cfg
!sed -i 's/steps=400000,450000/steps=9600,10800/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=6@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=6@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=6@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=33@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=33@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=33@' cfg/yolov3_training.cfg

In [ ]:
!echo "kick\npunch\nknee\nelbow\nclinch\nstance" > data/obj.names
!echo -e 'classes= 6\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3_multiclass_v2' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-11-30 08:24:18--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   288KB/s    in 8m 47s  

2020-11-30 08:33:07 (301 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
!unzip /mydrive/yolov3_multiclass_v2/images.zip -d data/obj

Archive:  /mydrive/yolov3_multiclass_v2/images.zip
  inflating: data/obj/2clinch1.png   
  inflating: data/obj/__MACOSX/._2clinch1.png  
  inflating: data/obj/2clinch1.txt   
  inflating: data/obj/2clinch2.png   
  inflating: data/obj/__MACOSX/._2clinch2.png  
  inflating: data/obj/2clinch2.txt   
  inflating: data/obj/2clinch3.png   
  inflating: data/obj/__MACOSX/._2clinch3.png  
  inflating: data/obj/2clinch3.txt   
  inflating: data/obj/2clinch4.png   
  inflating: data/obj/__MACOSX/._2clinch4.png  
  inflating: data/obj/2clinch4.txt   
  inflating: data/obj/2clinch5.png   
  inflating: data/obj/__MACOSX/._2clinch5.png  
  inflating: data/obj/2clinch5.txt   
  inflating: data/obj/2clinch6.png   
  inflating: data/obj/__MACOSX/._2clinch6.png  
  inflating: data/obj/2clinch6.txt   
  inflating: data/obj/2clinch7.png   
  inflating: data/obj/__MACOSX/._2clinch7.png  
  inflating: data/obj/2clinch7.txt   
  inflating: data/obj/2clinch8.png   
  inflating: data/obj/__MACOSX/._2clinch8.p

In [ ]:
#this grabs just the filepaths of the pics in the folder with the labeled images and puts them in a list
import glob
images_list_jpg = glob.glob("data/obj/*.jpg")
images_list_jpeg = glob.glob("data/obj/*.jpeg")
images_list_png = glob.glob("data/obj/*.png")
print(len(images_list_jpg), images_list_jpg)
print()
print(len(images_list_jpeg), images_list_jpeg)
print()
print(len(images_list_png), images_list_png)

171 ['data/obj/kick_sittichai-kicks-1024x576.jpg', 'data/obj/punch_Yokkao_41_Liam_Harrison_vs_Cristian_Faustino.jpg', 'data/obj/punch_images (1)6.jpg', 'data/obj/kick_muay-thai-techniques.jpg', 'data/obj/elbow_Ajh2eiURGqwdIi1hEhAp_file.jpg', 'data/obj/kick_220px-Roundhouse-kick.jpg', 'data/obj/elbow_944779295b12cf9d9eeac7cefdd0f5f8.jpg', 'data/obj/kick_3lowkickfeature-672x365.jpg', 'data/obj/kick_images (3).jpg', 'data/obj/punch_gettyimages-1159675648-1024x1024.jpg', 'data/obj/kick_s-l400.jpg', 'data/obj/punch_main-qimg-ee8742642fc61f73a1f8e6df920c983e.jpg', 'data/obj/kick_maxresdefault (8).jpg', 'data/obj/kick_hqdefault (3).jpg', 'data/obj/kick_Orono-Wor-Petchpun-1.jpg', 'data/obj/kick_maxresdefault (1).jpg', 'data/obj/elbow_buakaw-banchamek-gaetan-dambo.jpg', 'data/obj/kick_Muay-Thai-Moves-and-Techniques.jpg', 'data/obj/kick_103-aa-fp-mt-1.jpg', 'data/obj/elbow_main-qimg-9daca34bf95040196b059c7f512fa85f.jpg', 'data/obj/kick_maxresdefault (6).jpg', 'data/obj/elbow_main-qimg-2a8ccf2fab

In [ ]:
images_list = images_list_jpg + images_list_jpeg + images_list_png
len(images_list)

1156

In [ ]:
#Opens training.txt file and puts in the filepaths we found above
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

# Start training

In [ ]:
# Start the training. need to start from the beginning AGAIN bcs the .jpeg images were not being trained on. 
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show #(first run, using darknet weights and google colab pro)

Streaming output truncated to the last 5000 lines.
 total_bbox = 19513, rewritten_bbox = 0.005125 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.162446), count: 4, total_loss = 6.806389 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.272281), count: 6, total_loss = 9.815663 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, total_loss = 0.097667 
 total_bbox = 19523, rewritten_bbox = 0.005122 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.516091), count: 4, total_loss = 3.056539 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, total_loss = 0.102579 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, total_loss = 0.097002 
 total_bbox = 19527, rewritten_bbox = 0.005121 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: